In [10]:
import os
import cv2
import kagglehub

download_path = kagglehub.dataset_download("safabouguezzi/german-traffic-sign-detection-benchmark-gtsdb")

# Paths
image_dir = download_path + "/TrainIJCNN2013/TrainIJCNN2013"
annotation_file = download_path + "/gt.txt"
output_image_dir = "dataset/images/train"
output_label_dir = "dataset/labels/train"

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Read annotations
with open(annotation_file, "r") as f:
    lines = f.readlines()

# Parse and convert each line
for line in lines:
    parts = line.strip().split(";")
    img_name, x1, y1, x2, y2, class_id = parts
    x1, y1, x2, y2, class_id = map(int, (x1, y1, x2, y2, class_id))
    
    img_path = os.path.join(image_dir, img_name)
    if not os.path.exists(img_path):
        continue

    # Get image size
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Convert to YOLO format
    x_center = ((x1 + x2) / 2) / w
    y_center = ((y1 + y2) / 2) / h
    width = (x2 - x1) / w
    height = (y2 - y1) / h

    # Write to label file
    label_path = os.path.join(output_label_dir, img_name.replace(".ppm", ".txt"))
    with open(label_path, "a") as label_file:
        label_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # Copy image to output dir
    out_img_path = os.path.join(output_image_dir, img_name)
    if not os.path.exists(out_img_path):
        cv2.imwrite(out_img_path, img)


In [11]:
from PIL import Image
import os
from pathlib import Path

# Change this to your actual images folder
folder = 'dataset/images/train'

for ppm_path in Path(folder).glob('*.ppm'):
    with Image.open(ppm_path) as img:
        jpg_path = ppm_path.with_suffix('.jpg')
        img.convert('RGB').save(jpg_path)
        print(f"Converted {ppm_path} -> {jpg_path}")
        ppm_path.unlink()  # Optional: delete original .ppm


Converted dataset\images\train\00000.ppm -> dataset\images\train\00000.jpg
Converted dataset\images\train\00001.ppm -> dataset\images\train\00001.jpg
Converted dataset\images\train\00002.ppm -> dataset\images\train\00002.jpg
Converted dataset\images\train\00003.ppm -> dataset\images\train\00003.jpg
Converted dataset\images\train\00004.ppm -> dataset\images\train\00004.jpg
Converted dataset\images\train\00005.ppm -> dataset\images\train\00005.jpg
Converted dataset\images\train\00006.ppm -> dataset\images\train\00006.jpg
Converted dataset\images\train\00007.ppm -> dataset\images\train\00007.jpg
Converted dataset\images\train\00008.ppm -> dataset\images\train\00008.jpg
Converted dataset\images\train\00009.ppm -> dataset\images\train\00009.jpg
Converted dataset\images\train\00010.ppm -> dataset\images\train\00010.jpg
Converted dataset\images\train\00011.ppm -> dataset\images\train\00011.jpg
Converted dataset\images\train\00012.ppm -> dataset\images\train\00012.jpg
Converted dataset\images\

In [12]:
# bash: yolo detect train data=data.yaml model=yolov8n.pt epochs=10 imgsz=640

In [14]:
import os
import cv2
import kagglehub

# Download dataset
download_path = kagglehub.dataset_download("safabouguezzi/german-traffic-sign-detection-benchmark-gtsdb")

# Paths
image_dir = download_path + "/TestIJCNN2013/TestIJCNN2013Download"
output_image_dir = "dataset/images/test"

# Create output directory
os.makedirs(output_image_dir, exist_ok=True)

# Copy all .ppm images from test set to output directory
for img_name in os.listdir(image_dir):
    if img_name.endswith(".ppm"):
        img_path = os.path.join(image_dir, img_name)
        out_img_path = os.path.join(output_image_dir, img_name)
        
        # Read and save image
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imwrite(out_img_path, img)

In [15]:
from PIL import Image
import os
from pathlib import Path

# Change this to your actual images folder
folder = 'dataset/images/test'

for ppm_path in Path(folder).glob('*.ppm'):
    with Image.open(ppm_path) as img:
        jpg_path = ppm_path.with_suffix('.jpg')
        img.convert('RGB').save(jpg_path)
        print(f"Converted {ppm_path} -> {jpg_path}")
        ppm_path.unlink()  # Optional: delete original .ppm


Converted dataset\images\test\00000.ppm -> dataset\images\test\00000.jpg
Converted dataset\images\test\00001.ppm -> dataset\images\test\00001.jpg
Converted dataset\images\test\00002.ppm -> dataset\images\test\00002.jpg
Converted dataset\images\test\00003.ppm -> dataset\images\test\00003.jpg
Converted dataset\images\test\00004.ppm -> dataset\images\test\00004.jpg
Converted dataset\images\test\00005.ppm -> dataset\images\test\00005.jpg
Converted dataset\images\test\00006.ppm -> dataset\images\test\00006.jpg
Converted dataset\images\test\00007.ppm -> dataset\images\test\00007.jpg
Converted dataset\images\test\00008.ppm -> dataset\images\test\00008.jpg
Converted dataset\images\test\00009.ppm -> dataset\images\test\00009.jpg
Converted dataset\images\test\00010.ppm -> dataset\images\test\00010.jpg
Converted dataset\images\test\00011.ppm -> dataset\images\test\00011.jpg
Converted dataset\images\test\00012.ppm -> dataset\images\test\00012.jpg
Converted dataset\images\test\00013.ppm -> dataset\

In [ ]:
# bash: yolo detect predict model=runs/detect/train/weights/best.pt source=dataset/images/test